# SCN Model Training and Testing

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from gen_utils.sr_gen import sr_gen # Custom class for image generation/organization

In [2]:
class SCN(nn.Module):
    def __init__(self,sy,sg, train=True):
        super().__init__()
        C = 5
        L = 5

        Dx = torch.normal(0,1, size = (25,128))
        Dy = torch.normal(0,1, size = (100,128))
        I = torch.eye(128)

        self.conv = nn.Conv2d(1,100,9, bias = False, stride =1, padding = 6, padding_mode='reflect')
        self.mean2 = nn.Conv2d(1,1,13, bias = False, stride = 1, padding = 6, padding_mode='reflect')
        self.diffms = nn.Conv2d(1,25,9, bias=False, stride = 1, padding=6, padding_mode='reflect')

        self.wd = nn.Conv2d(100,128,1,bias = False, stride = 1)
        self.usd1 = nn.Conv2d(128, 128, 1, bias = False, stride=1)
        self.ud = nn.Conv2d(128,25,1,bias=False,stride=1)
        self.addp = nn.Conv2d(16,1,1, bias = False, stride = 1)

        if train: #If you are currently training the model
            self.mean2.weight = torch.nn.Parameter(self.create_gaus(13), requires_grad = False)
            self.diffms.weight = torch.nn.Parameter(self.create_diffms(9,5),requires_grad=False)
            self.wd.weight = torch.nn.Parameter(self.expand_params(C*Dy.T), requires_grad=True)
            self.usd1.weight = torch.nn.Parameter(self.expand_params(I - torch.matmul(Dy.T,Dy)), requires_grad=True)
            self.ud.weight = torch.nn.Parameter(self.expand_params((1/(C*L))*Dx), requires_grad=True)
            self.addp.weight = torch.nn.Parameter(torch.ones(1,16,1,1)*0.06, requires_grad=True)

        else:
            self.conv.weight = torch.nn.Parameter(torch.ones(100,1,9,9),requires_grad=False)
            self.mean2.weight = torch.nn.Parameter(self.create_gaus(13),requires_grad=False)
            self.diffms.weight = torch.nn.Parameter(self.create_diffms(9,5),requires_grad=False)
            self.wd.weight = torch.nn.Parameter(self.expand_params(C*Dy.T),requires_grad=False)
            self.usd1.weight = torch.nn.Parameter(self.expand_params(I - torch.matmul(Dy.T,Dy)),requires_grad=False)
            self.ud.weight = torch.nn.Parameter(self.expand_params((1/(C*L))*Dx),requires_grad=False)
            self.addp.weight = torch.nn.Parameter(torch.ones(1,16,1,1)*0.06,requires_grad=False)


    def forward(self, x, k, sy=9, sg=5):
        #print(f'input: {x.min()}-{x.max()}')
        x = x+0.1

        im_mean = self.mean2(x)
        # print(f'im_mean shape {im_mean.shape}')
        diffms = self.diffms(x)
        # print(f'diffms shape: {diffms.shape}')

        n, c, h, w = x.shape
        # y = torch.zeros(n, 100, h-8, w-8)
        x = self.conv(x)
        # print(f'post conv shape {x.shape}')
        #print(f'conv max {x.max()}')
        #x=x+1

        x = x/torch.linalg.vector_norm(x, ord=2, dim=1, keepdim=True)
        # print(f'post vector norm shape: {x.shape}')
        #print(f'postnorm max {x.max()}')

        x = self.wd(x)
        #print(f'conv wd {x.max()}')
        z = self.ShLU(x,1)
        #print(f'conv SHLU {x.max()}')

        # Go through LISTA
        for i in range(k):
            z = self.ShLU(self.usd1(z)+x,1)

        x = self.ud(z)
        #print(f'ud max {x.max()}')
        # print(f'post ud shape {x.shape}')
        x = (x/torch.linalg.vector_norm(x, ord=2, dim=1, keepdim=True))*torch.linalg.vector_norm(diffms, ord=2, dim=1, keepdim=True)*1.1
        # print(f'prereassembled x shape {x.shape}')
        x = self.reassemble2(x,im_mean,4)
        # print(f'reassembled x shape {x.shape}')
        x = self.addp(x)
        #print(f'x.reassemble.max = {x.max()}')
        x = x+im_mean

        #print(f'output: {x.min()}-{x.max()}')

        return x

    def reassemble2(self, x, im_mean, patch_size):
        img = im_mean
        s, c, h, w = img.shape
        
        # img_stack=torch.zeros(s,25,h,w)
        img_stack=torch.zeros(s,16,h,w)
        
        #go through every sample and reassemble the image
        for q in range(x.shape[0]):
            filt = 0
            for ii in range(patch_size-1, -1, -1):
                for jj in range(patch_size-1, -1, -1):
                    img_stack[q,filt,:,:] = x[q,filt,jj:(jj+h), ii:(ii+w)]
                    filt+=1
        
        return img_stack
    
    def create_diffms(self, kern_size, sy=5):
        diffms = torch.zeros(sy**2,1,kern_size,kern_size)
        
        neg = -1*(1/(sy**2))
        pos = 1+neg
        
        border = int((kern_size-sy)/2)
        base = torch.zeros(sy,sy)+neg
        cnt=0
        
        for i in range(sy**2):
            base = torch.zeros(sy**2)+neg
            base[cnt]=pos
            diffms[i,0,border:(kern_size-border),border:(kern_size-border)] = base.reshape([sy,sy])
            cnt+=1
        return diffms
    
    
    def create_gaus(self, kern_size, sy=9,std=2.15):
        n = torch.arange(0,sy)-(sy-1.0)/2.0
        sig2 = 2 * std * std
        gkern1d = torch.exp(-n ** 2 / sig2)
        gkern1d = gkern1d/torch.sum(gkern1d)
        #print(gkern1d.shape)
        gkern2d = torch.outer(gkern1d, gkern1d)
    

        # Wrap in zeros, if kern_size > sy
        gaussian_filter = torch.zeros(1,1,kern_size,kern_size)
        border = int((kern_size-sy)/2)
        gaussian_filter[0,0,border:(kern_size-border),border:(kern_size-border)] = gkern2d#(sy,std=std)
        #print(gaussian_filter.shape)
        return gaussian_filter
        
    
    def fixed_positions(self, tens, mult, sg):
        f, _ , h, w = tens.shape
        new_filt = torch.zeros(f*mult, 1, sg,sg)
        cnt = 0
        filt = 0
        
        for filt in range(f):
            for j in range((sg-w)+1):
                for i in range((sg-h)+1):
                    new_filt[cnt,0,i:i+h,j:j+w] = tens[filt]
                    cnt+=1
        return new_filt
    
    def expand_params(self,tens):
        return torch.unsqueeze(torch.unsqueeze(tens,2),3)
    
    def ShLU(self,a, th):
        return torch.sign(a)*torch.maximum(abs(a)-th, torch.tensor(0))

# Set Optimization Parameters

In [3]:
net = SCN(9,5,train=True)

#net.load_state_dict(torch.load('./MRI_save_29.p'))

criterion = nn.MSELoss()
optimizer = optim.SGD(
    [
        {"params": net.addp.parameters()},#, "lr": 0.0002, "momentum": 0.00005},
        {"params": net.conv.parameters()},#, "lr": 0.0003, "momentum": 0.0001},
        {"params": net.wd.parameters()},
        {"params": net.usd1.parameters()},
        {"params": net.ud.parameters()},
    ],
    #lr=0.0001, momentum=0.0001
    lr=0.00007, momentum = 0.0001
)

## Generate Data for Training

In [4]:
sr_train = sr_gen('./data/train/GT_corr/','./data/train/HR_corr_patches/','./data/train/LR_corr_patches/')

In [5]:
temp = sr_train.get_template()
temp["patch"]=44
temp["step"]=20
temp["translation_x"]=10
temp["translation_y"]=10
temp["rotation"] = 180
temp["scale"] = 1
sr_train.save_template(temp)

sr_train.run(clear=True)

Clearing existing output directories


## Create Training Dataset and Dataloader

In [6]:
# TODO: There's not reasing I can't combine the Dataset class and my custom class into one thing

class Dataset(torch.utils.data.Dataset):
    def __init__(self, sr_class):
        self.sr_class = sr_class

        # In case I forget to run match_altered before pulling the class
        if not sr_class.HR_files:
            sr_class.match_altered(update=True)

    def __len__(self):
        return len(self.sr_class.HR_files)

    def __getitem__(self, index):
        Y, X = self.sr_class.load_image_pair(index)
        X = torch.unsqueeze(torch.tensor(X, dtype=torch.float32),0)
        Y = torch.unsqueeze(torch.tensor(Y, dtype=torch.float32),0)

        return X, Y

In [7]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 3}

training_set = Dataset(sr_train)
training_generator = torch.utils.data.DataLoader(training_set, **params)

## Training Loop

In [8]:
from tqdm import tqdm
import time
# Loop over epochs

max_epochs = 40
save_rate = 5 #save a version of the model every 5 epochs
epoch_adjust = 0 #how much to add to the saved files in order to not overwrite
save_prefix = "./MRI_reflect_pad_save_"

mean_loss = []

for epoch in tqdm(range(max_epochs)):
    losses = []

    ###### Test running this code where each epoch a new set of random images is made
    sr_train.run(clear=True)

    training_set = Dataset(sr_train)
    training_generator = torch.utils.data.DataLoader(training_set, **params)
    ######


    # Training
    count = 0
    for inp, goal in training_generator:
        optimizer.zero_grad()

        output = net(inp,2) # the 2 is the number of iterations in the LISTA network
        output = torch.clamp(output, 0, 255)

        loss = criterion(output,goal)
        loss.backward()
        optimizer.step()
        #print(f'loss = {loss.item()}')
        losses.append(loss.item())
        #print(f'mini-batch # {count}, mean loss = {sum(losses)/len(losses)}')
        count = count+1

    if (epoch % save_rate == 0) or epoch == (max_epochs-1):
        torch.save(net.state_dict(), f'{save_prefix}{epoch+epoch_adjust}.p')
    print(f'\n\n epoch {epoch}, loss mean: {sum(losses)/len(losses)}, loss: {min(losses)}-{max(losses)}\n')
    mean_loss.append(sum(losses)/len(losses))

    # Give computer time to cool down
    time.sleep(90)

  0%|          | 0/40 [00:00<?, ?it/s]

Clearing existing output directories


 epoch 0, loss mean: 98.32311543551359, loss: 54.4627799987793-167.3396759033203



  2%|▎         | 1/40 [03:28<2:15:19, 208.19s/it]

Clearing existing output directories


 epoch 1, loss mean: 51.35435017672452, loss: 42.5849609375-68.5630874633789



  5%|▌         | 2/40 [06:57<2:12:15, 208.83s/it]

...

 95%|█████████▌| 38/40 [2:07:21<06:41, 200.89s/it]

Clearing existing output directories


 epoch 38, loss mean: 25.892704963684082, loss: 23.212108612060547-29.126216888427734



 98%|█████████▊| 39/40 [2:10:42<03:20, 200.83s/it]

Clearing existing output directories


 epoch 39, loss mean: 26.585525946183637, loss: 21.103351593017578-30.358949661254883



100%|██████████| 40/40 [2:14:03<00:00, 201.10s/it]


In [24]:
print(mean_loss)

[24.94681098244407]


## Testing Loop

In [10]:
# Generate testing data
sr_test = sr_gen('./data/test/GT_corr/','./data/test/HR_corr/','./data/test/LR_corr/')

In [11]:
temp = sr_test.get_template()
temp["patch"]=False #44
# temp["step"]=20
# temp["rotation"] = 180
temp["scale"] = 1
temp["rotation"]=180
sr_test.save_template(temp)

sr_test.run(clear=True)

Clearing existing output directories
Saving image: 155_rot91
Saving image: 140_rot67
Saving image: 145_rot-144
Saving image: 160_rot-1
Saving image: 150_rot150


### OPTIONAL: Load previously trained model

In [15]:
net = SCN(9,5,train=False) #Switch to True if you want to keep training
net.load_state_dict(torch.load('./MRI_reflect_pad_save_39.p'))

<All keys matched successfully>

In [16]:
# Load matched images
im_hr, im_lr = sr_test.match_altered(update = True, paths=True)

save_pred = False # Whether to save the images created by the network during testing
save_dir = "./"
if save_pred:
    os.makedirs(save_dir, exist_ok=True)


with torch.no_grad():
    for i in im_hr: #range(len(im_hr)):

        # Load in image information
        im_h, im_l = sr_test.load_image_pair(i)

        # Take low resolution and upscale using bicubic interpolation
        # (which has already been done due to the image generation process)
        # Thus im_l is the bicubic interpolation to compare to...

        # Use SR model on low resolution image
        im_h_sr = net(torch.unsqueeze(torch.unsqueeze(torch.tensor(im_l, dtype=torch.float32),0),0),2)

        # Calculate PSNR for bicubic
        im_l = np.rint( np.clip(im_l, 0, 255))
        im_h = np.rint( np.clip(im_h, 0, 255))
        diff = im_l - im_h
        rmse = np.sqrt((diff**2).mean())
        psnr = 20*np.log10(255.0/rmse)

        print(f'bicubic evaluation for {i}: rms={rmse}, psnr={psnr}')

        # Calculate PSNR for SR
        im_h_sr = np.rint( np.clip(im_h_sr, 0, 255))
        im_h = np.rint( np.clip(im_h, 0, 255))
        diff = im_h_sr - im_h
        rmse = np.sqrt((diff**2).mean())
        psnr = 20*np.log10(255.0/rmse)
        print(f'SR evaluation for {i}: rms={rmse}, psnr={psnr}')

        if save_pred:
            img_name = os.path.splitext(os.path.basename(i))[0]
            Image.fromarray(np.rint(im_h_sr).astype(np.uint8)).save(f"{save_dir}/{img_name}_SR.png")


HR and LR file locations updated
bicubic evaluation for ./data/test/HR_corr/155_rot91.png: rms=4.91015625, psnr=34.30889736119694
SR evaluation for ./data/test/HR_corr/155_rot91.png: rms=4.818031311035156, psnr=34.473411560058594
bicubic evaluation for ./data/test/HR_corr/145_rot-144.png: rms=4.546875, psnr=34.976543308638696
SR evaluation for ./data/test/HR_corr/145_rot-144.png: rms=4.458677768707275, psnr=35.14668273925781
bicubic evaluation for ./data/test/HR_corr/140_rot67.png: rms=3.361328125, psnr=37.60058542164452
SR evaluation for ./data/test/HR_corr/140_rot67.png: rms=3.3172366619110107, psnr=37.715274810791016
bicubic evaluation for ./data/test/HR_corr/150_rot150.png: rms=5.20703125, psnr=33.79899992663891
SR evaluation for ./data/test/HR_corr/150_rot150.png: rms=5.131485939025879, psnr=33.925941467285156
bicubic evaluation for ./data/test/HR_corr/160_rot-1.png: rms=4.40234375, psnr=35.257124593993964
SR evaluation for ./data/test/HR_corr/160_rot-1.png: rms=4.329180717468262,